In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, lit, avg, when, broadcast

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
jobs_df = spark.table("database.jobs")
popu_df = spark.table("database.population")
jobs_df = jobs_df.filter(col("indicador_name").like("%Literacy rate%"))
jobs_df = jobs_df.drop(col("pais_cod"), col("indicador_code"))
popu_df = popu_df.filter(col("cod_indicador").like("%SP.POP.TOTL%"))
popu_df = popu_df.drop(col("cod_pais"), col("cod_indicador"), col("nome_indicador"), col("Country_name"), col("Country_code"), col("Indicator_name"), col("Indicator_code"))
popu_df = popu_df.withColumnRenamed("valor", "valorI")
popu_df = popu_df.withColumnRenamed("nome_pais", "pais_name")
popu_df = popu_df.withColumn("ano", col("ano").cast(IntegerType()))

In [3]:
paises = ["%East Asia & Pacific", "%European Union%", "%Sub-Saharan Africa"]
anos = ["1990", "1995", "2000", "2005", "2010", "2015"]

jobs_df = jobs_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2])
).filter(
    col("ano").isin(anos) &
    col("valor").isNotNull()
)

popu_df = popu_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2])
).filter(
    col("ano").isin(anos) &
    col("valorI").isNotNull()
)

In [4]:
literacia_df = popu_df.join(jobs_df, ["pais_name", "ano"], "left")
literacia_df = literacia_df.withColumn("valorI", ((col("valor") / 100) * (col("valorI") / 2)).cast("int"))
literacia_df = literacia_df.withColumnRenamed("pais_name", "regiao")

In [5]:
literacia_df = literacia_df.filter(
    col("indicador_name").like("%adult%")
)

literacia_df = literacia_df.withColumn("genero",
    when(col("indicador_name").like("% female%"), "F")
    .when(col("indicador_name").like("% male%"), "M")
)

literacia_df = literacia_df.filter(col("genero").isNotNull())

literacia_df = literacia_df.drop(col("indicador_name"))

In [6]:
literacia_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaLiteracia/")

In [7]:
spark.sql(
    """
    SELECT * FROM database.LiteraciaGold
    """
).show()

+-------------------+----+--------+----------+------+
|             regiao| ano|   valor|    valorI|genero|
+-------------------+----+--------+----------+------+
|East Asia & Pacific|1990|74.78902| 679924732|     F|
|East Asia & Pacific|1995| 78.7219| 763953853|     F|
|East Asia & Pacific|2000| 87.8352| 899497743|     F|
|East Asia & Pacific|2005|90.26026| 962408053|     F|
|East Asia & Pacific|2010|93.03505|1028132028|     F|
|East Asia & Pacific|2015|93.93781|1077704848|     F|
|East Asia & Pacific|1990| 89.0924| 809960187|     M|
|East Asia & Pacific|1995|91.18186| 884871096|     M|
|East Asia & Pacific|2000|95.04204| 973301149|     M|
|East Asia & Pacific|2005|96.03638|1023996397|     M|
|East Asia & Pacific|2010|97.08977|1072940791|     M|
|East Asia & Pacific|2015|97.30305|1116312618|     M|
|     European Union|1990|97.24331| 204361123|     F|
|     European Union|1995|97.62325| 208041071|     F|
|     European Union|2000|98.16005| 210721387|     F|
|     European Union|2005|98

In [8]:
spark.stop()